# Lesson 2: Implementing Atomic Transactions with Watch

# Introduction to `watch` in Redis

Welcome back! You’ve now learned how to build and execute basic transactions in Redis. This next lesson takes you a step further by introducing the `watch` command. This command will help you implement more controlled and conditional transactions. It is essential for scenarios where you need to monitor certain keys and ensure that operations are completed only when specific conditions are met.

## What You'll Learn
In this unit, you will delve into the functionality of the `watch` command in Redis. Here’s a quick overview of what you will learn:

- **Setting Up `watch`**: Understand the importance of monitoring keys to control transaction execution.
- **Implementing Conditional Updates**: Write functions that use `watch` to implement safer and more conditional updates to your Redis data.

Let's take a look at a practical example of how to use `watch` in your code.

```python
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

def update_balance(user_id, increment):
    with client.pipeline() as pipe:
        while True:
            try:
                pipe.watch(f'balance:{user_id}')
                balance = int(pipe.get(f'balance:{user_id}') or 0)
                pipe.multi()
                pipe.set(f'balance:{user_id}', balance + increment)
                pipe.execute()
                break
            except redis.WatchError as e:
                print(f"Retrying transaction: {e}")
                continue

client.set('balance:1', 100)
update_balance(1, 50)

value = client.get('balance:1').decode('utf-8')
print(f"Updated balance for user 1: {value}")
```

In this example, we start by **watching** the `balance:{user_id}` key to monitor changes. If another client modifies the value before executing the transaction, the `pipe.execute()` will fail, and the transaction will retry. This ensures that your balance updates are consistent.

### Breakdown of Each Step:

1. **Define the Function**: The `update_balance` function takes `user_id` and `increment` as arguments.
2. **Create a Pipeline**: We use `client.pipeline()` to execute multiple commands in a single transaction.
3. **Monitor Key**: Inside the `while` loop, we use the `watch` command to monitor the `balance:{user_id}` key, ensuring that no other client modifies it during the transaction.
4. **Get the Current Balance**: We retrieve the current balance value using `pipe.get()` and set it to `0` if it doesn't exist.
5. **Start the Transaction**: The `pipe.multi()` command starts the transaction block, allowing multiple commands to be executed atomically.
6. **Update the Balance**: We update the balance by adding the `increment` value to the current balance.
7. **Execute the Transaction**: The `pipe.execute()` command executes the transaction.
8. **Handle Conflicts**: If another client changes the balance key before the transaction is executed, a `redis.WatchError` exception is raised, and the transaction is retried using the `continue` statement.

### How the Function is Used:

1. We set the initial balance for user 1 to `100`.
2. We call the `update_balance` function with `user_id=1` and `increment=50` to increase the balance by 50.
3. Finally, we retrieve the updated balance for user 1 and print it.

## Why It Matters

Mastering the `watch` command is crucial for several reasons:

- **Optimized Data Integrity**: Using `watch` ensures that actions only occur if specific conditions are met, allowing for safer updates.
- **Conditional Logic**: You can tailor your Redis transactions to proceed only when specific keys maintain expected values, adding sophistication and precision to your operations.
- **Error Handling**: `watch` helps in avoiding conflicts and managing errors when multiple clients try to update the same data.

Using `watch` effectively enables you to write more robust and reliable applications, safeguarding against potential race conditions and ensuring that concurrent updates do not interfere with each other.

Ready to get hands-on and explore further? Let’s move on to the practice section and apply these commands in various scenarios to solidify your understanding.

## Watch and Update User Balance

Great job learning how to implement watch in Redis!

Let's run the code you saw in the lesson and pay careful attention to the output since it will help you better understand this construct.

Using the watch command in Redis allows you to monitor specific keys and ensure that transactions are executed only if the monitored keys remain unchanged. This is crucial for maintaining data consistency when multiple clients access and modify the same data.

In this example, we'll monitor and update the user balance in a Redis database:

A function update_balance updates the balance of a specific user.
The watch command monitors the user's balance key.
If the key is modified by another client before the transaction completes, the transaction is retried.
The multi method marks the beginning of the transaction block. After calling multi, subsequent commands are queued up instead of being executed immediately.
The execute method sends all queued commands within the transaction to be executed atomically.
Here's the code in action. Run it and observe how the balance is updated correctly, even if multiple clients attempt to modify the same key simultaneously.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

def update_balance(user_id, increment):
    with client.pipeline() as pipe:
        while True:
            try:
                pipe.watch(f'balance:{user_id}')
                balance = int(pipe.get(f'balance:{user_id}') or 0)
                pipe.multi()
                pipe.set(f'balance:{user_id}', balance + increment)
                pipe.execute()
                break
            except redis.WatchError:
                continue

client.set('balance:1', 100)
update_balance(1, 50)

value = client.get('balance:1').decode('utf-8')
print(f"Updated balance for user 1: {value}")
```

The code provided demonstrates how to use the `watch` command in Redis to ensure transactional consistency. Here's a breakdown of how the code functions:

### Overview
- **Watch Command**: Monitors the key (`balance:{user_id}`) to ensure no other client modifies the balance during the transaction.
- **Transaction Handling**: If the key is modified by another client before the transaction completes, it retries the entire operation. This ensures that the balance update is consistent.
- **Pipeline**: A Redis pipeline is used for grouping multiple commands and sending them as a single batch for efficiency.

### Detailed Breakdown
1. **Redis Client Setup**:
   - A connection to Redis is established using the `redis.Redis()` client.
   - The Redis server is expected to run locally (`localhost`) on port `6379`.

2. **Function `update_balance(user_id, increment)`**:
   - This function is responsible for safely updating a user's balance by the specified `increment`.

3. **Using the `watch` Command**:
   - Inside the `while` loop, the `pipe.watch()` command watches the balance key for the specified user.
   - The current balance is fetched, or if it doesn't exist, it defaults to `0`.

4. **Start the Transaction**:
   - After calling `pipe.multi()`, any subsequent Redis commands are queued for execution.

5. **Updating the Balance**:
   - The new balance (old balance + increment) is set with the `pipe.set()` command.

6. **Atomic Execution**:
   - `pipe.execute()` executes the queued commands atomically, ensuring no external modifications occur in between. If the key was modified during the transaction, a `redis.WatchError` is raised, and the function retries.

7. **Test Case**:
   - The balance for `user 1` is initially set to `100` using `client.set()`.
   - `update_balance(1, 50)` increments the balance by `50`.
   - The updated balance is printed to the console.

### Key Learning Points
- **Transaction Safety**: The `watch` and transaction mechanism in Redis prevents race conditions by ensuring that no other client can modify the watched key during the update.
- **Retry Logic**: The code retries if the key changes, ensuring consistency and successful update even in concurrent environments.

### Example Output
```
Updated balance for user 1: 150
```

This example demonstrates how Redis can be used in real-world scenarios where multiple clients may attempt to update the same data, ensuring safe and consistent updates.

## Updating User Balance with Redis

Great job so far!

Let's run the code you saw in the lesson and closely observe the output to solidify your understanding.

Using the watch command in Redis allows you to monitor specific keys and ensure that transactions are executed only if the monitored keys remain unchanged. This is crucial for maintaining data consistency when multiple clients access and modify the same data.

In this example, we'll monitor and update the user's score in a Redis database:

A function update_score updates the score of a specific user by 1. This method is called only once.
The watch command monitors the user's score key.
If the key is modified by another client before the transaction completes, the transaction is retried.
A separate thread continuously updates the score to simulate concurrent modifications. The value is updated 50 times from the thread.
Run the following code and observe how the score is updated correctly, even if multiple clients attempt to modify the same key simultaneously.

Note, that the result of the code will be different each time you run it due to the concurrent modifications - sometimes the transaction will be retried and execute successfully after a few attempts.

```py
import threading
import redis
import time

client = redis.Redis(host='localhost', port=6379, db=0)

def update_score(user_id, points):
    with client.pipeline() as pipe:
        while True:
            try:
                pipe.watch(f'score:{user_id}')
                score = int(pipe.get(f'score:{user_id}') or 0)
                pipe.multi()
                pipe.set(f'score:{user_id}', score + points)
                pipe.execute()
                print(f'Set value in transaction: {score + points}')
                break
            except redis.WatchError as e:
                print(f'Error: {e} Retrying...')
                continue

def update_score_continuously(user_id, points, iterations):
    for i in range(iterations):
        client.incrby('score:1', points)
        print('Incremented!')
        time.sleep(0.00001)

client.set('score:1', 150)

# Create and start a thread to update the score continuously
thread = threading.Thread(target=update_score_continuously, args=(1, 50, 50))
thread.start()

update_score(1, 1)

value = client.get('score:1').decode('utf-8')

print(f"Updated score for user 1: {value}")

# Wait for the thread to finish
thread.join()

final_value = client.get('score:1').decode('utf-8')
print(f"Final updated score for user 1: {final_value}")
```



## Abort Transaction on Negative Balance

Great job so far!

Now, let's add some complexity. Modify the existing update_balance function to accept a second value user ID as an argument. This user ID will be used to update the balance for the specified user.


```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)


# TODO: Modify the method to get second parameter user_id
def update_balance(increment):
    with client.pipeline() as pipe:
        while True:
            try:
                # TODO: Modify the watch command to monitor the balance key for any user ID instead of just user 1
                pipe.watch(f'balance:1')
                balance = int(pipe.get(f'balance:{user_id}') or 0)
                pipe.multi()
                new_value = balance + increment
                pipe.set(f'balance:{user_id}', new_value)
                pipe.execute()
                break
            except redis.WatchError:
                continue

client.set('balance:1', 100)
client.set('balance:15', 70)

# TODO: Modify the call to update_balance to pass the user ID as an argument for both users
update_balance(100)
update_balance(10)

value = client.get('balance:1').decode('utf-8')
print(f"Updated balance for user 1: {value}")
value = client.get('balance:15').decode('utf-8')
print(f"Updated balance for user 15: {value}")


```

To add support for a second user and allow the `update_balance` function to update the balance for any user, we need to pass the `user_id` as an argument to the function. Additionally, we will modify the `watch` command to monitor the balance of the specified user.

Here’s how the updated code will look:

### Updated Code

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

# Modified method to accept a second parameter user_id
def update_balance(user_id, increment):
    with client.pipeline() as pipe:
        while True:
            try:
                # Watch the balance key for the specified user ID
                pipe.watch(f'balance:{user_id}')
                balance = int(pipe.get(f'balance:{user_id}') or 0)
                
                # Abort transaction if balance would become negative
                new_value = balance + increment
                if new_value < 0:
                    print(f"Transaction aborted: balance cannot go negative for user {user_id}")
                    pipe.unwatch()
                    break

                # Start the transaction
                pipe.multi()
                pipe.set(f'balance:{user_id}', new_value)
                pipe.execute()
                print(f"Balance updated for user {user_id}: {new_value}")
                break
            except redis.WatchError:
                print(f"WatchError: Retrying transaction for user {user_id}")
                continue

# Set initial balances for both users
client.set('balance:1', 100)
client.set('balance:15', 70)

# Modify the call to update_balance to pass the user ID for both users
update_balance(1, 100)   # Update balance for user 1 by +100
update_balance(15, 10)   # Update balance for user 15 by +10

# Try an update that would cause a negative balance
update_balance(15, -100)  # This will abort because the balance would go negative

# Fetch and print final balances
value_1 = client.get('balance:1').decode('utf-8')
print(f"Final balance for user 1: {value_1}")
value_15 = client.get('balance:15').decode('utf-8')
print(f"Final balance for user 15: {value_15}")
```

### Key Changes
1. **Function Modification**:
   - The `update_balance` function now accepts two parameters: `user_id` and `increment`.
   - The `watch` command now monitors the `balance:{user_id}` key, making the function flexible for any user.

2. **Transaction Abort for Negative Balances**:
   - Before executing the transaction, the new balance is calculated (`new_value = balance + increment`).
   - If the new balance is negative, the transaction is aborted by calling `pipe.unwatch()`, and an error message is printed.

3. **Multiple Calls**:
   - We update the balance for two users (`user_id=1` and `user_id=15`).
   - A final update attempts to decrement the balance for `user 15` by more than the current balance, which triggers the abort mechanism.

### Example Output

```
Balance updated for user 1: 200
Balance updated for user 15: 80
Transaction aborted: balance cannot go negative for user 15
Final balance for user 1: 200
Final balance for user 15: 80
```

### Explanation

- The `update_balance` function successfully updates the balance for both `user 1` and `user 15`.
- The transaction for `user 15` to subtract `100` is aborted because it would result in a negative balance.
- The final balances are printed, reflecting the successful updates.

## Update Likes with Redis Transactions

Now that you've learned how to update user balances in Redis using watch, let's apply the same concept to another scenario.

In this practice, we're focusing on updating the number of likes a post has received. The update_likes function will ensure that the likes for a given post_id key are updated only if it has not been modified by another client during the transaction. If it has been modified, the transaction will retry until it succeeds.

Fill in the missing parts of the provided code to implement the watch functionality correctly.

```PY
import redis

client = redis.Redis(host='localhost', port=6379, db=0)


def update_likes(post_id, increment):
    with client.pipeline() as pipe:
        while True:
            try:
                # TODO: Watch the 'likes:{post_id}' key
                
                # TODO: Retrieve the current number of likes for the 'likes:{post_id}' key
                
                # TODO: Begin the transaction with multi() method
                
                # TODO: Update the number of likes for the 'likes:{post_id}' key by setting value to likes + increment
                
                # TODO: Execute the transaction
                
                break
            except redis.WatchError:
                continue


client.set('likes:99', 150)
update_likes(99, 15)

value = client.get('likes:99').decode('utf-8')
print(f"Updated likes for post 99: {value}")
```

Here’s the complete code for updating the number of likes for a post in Redis, using the `watch` command to ensure that the transaction succeeds only if the `likes` key has not been modified by another client during the transaction:

### Completed Code

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)


def update_likes(post_id, increment):
    with client.pipeline() as pipe:
        while True:
            try:
                # Watch the 'likes:{post_id}' key
                pipe.watch(f'likes:{post_id}')
                
                # Retrieve the current number of likes for the 'likes:{post_id}' key
                likes = int(pipe.get(f'likes:{post_id}') or 0)
                
                # Begin the transaction with multi() method
                pipe.multi()
                
                # Update the number of likes for the 'likes:{post_id}' key by setting value to likes + increment
                pipe.set(f'likes:{post_id}', likes + increment)
                
                # Execute the transaction
                pipe.execute()
                
                print(f"Transaction successful: {likes + increment} likes for post {post_id}")
                break
            except redis.WatchError:
                # If another client modified the key, retry the transaction
                print(f"WatchError: Retrying transaction for post {post_id}")
                continue


# Set initial number of likes for post 99
client.set('likes:99', 150)

# Update the number of likes for post 99 by adding 15
update_likes(99, 15)

# Fetch and print the updated number of likes
value = client.get('likes:99').decode('utf-8')
print(f"Updated likes for post 99: {value}")
```

### Key Points

1. **Watch Command**:
   - `pipe.watch(f'likes:{post_id}')` monitors the `likes:{post_id}` key to ensure that the transaction only proceeds if no other client modifies this key during the process.

2. **Get the Current Number of Likes**:
   - `likes = int(pipe.get(f'likes:{post_id}') or 0)` retrieves the current number of likes. If no value exists, it defaults to `0`.

3. **Transaction Start**:
   - `pipe.multi()` marks the beginning of the transaction block, where subsequent commands are queued.

4. **Update Likes**:
   - `pipe.set(f'likes:{post_id}', likes + increment)` updates the number of likes by adding the specified `increment`.

5. **Execute the Transaction**:
   - `pipe.execute()` runs the queued commands atomically.

6. **Retry on Failure**:
   - If the key is modified during the transaction by another client, a `redis.WatchError` is raised, and the transaction is retried.

### Example Output

```
Transaction successful: 165 likes for post 99
Updated likes for post 99: 165
```

This code successfully ensures that the likes for a post are updated in a safe and consistent manner, even in the presence of concurrent modifications by other clients. The `watch` command guarantees that the transaction either succeeds or retries until it can be applied correctly.

## Update Points with Redis Transactions

You’ve done well learning the watch command in Redis for enforcing conditional transactions.

Now, let's put everything together in this practice task. You'll write a function to update user points in a Redis database. Use watch to ensure the transaction succeeds only if the monitored key remains unchanged. If another client updates the key during the transaction, it should retry until it works.

```py
import redis

client = redis.Redis(host='localhost', port=6379, db=0)

# TODO: Define a function `update_points` that takes two parameters: `user_id` and `points`
    # TODO: Inside the function, implement the following in a loop:
    # - Create a pipeline using `client.pipeline()`
    # - Use the `pipe.watch` method to monitor the key `points:{user_id}`
    # - Retrieve the current points using `pipe.get`
    # - Use the `pipe.multi` method to begin the transaction
    # - Update the points by adding the given `points` to the current points
    # - Execute the transaction using `pipe.execute`
    # - Handle the `redis.WatchError` exception to retry the transaction if necessary

client.set('points:2', 200)
update_points(2, 30)

value = client.get('points:2').decode('utf-8')
print(f"Updated points for user 2: {value}")
```

Here is the complete implementation for the `update_points` function using Redis transactions with the `watch` command. The function will keep retrying the transaction if another client modifies the key during the process.

```py
import redis

# Initialize Redis client
client = redis.Redis(host='localhost', port=6379, db=0)

# Function to update points
def update_points(user_id, points):
    key = f'points:{user_id}'  # The key to monitor and update
    
    while True:  # Retry loop in case of a WatchError
        try:
            # Start pipeline and watch the key
            with client.pipeline() as pipe:
                pipe.watch(key)
                
                # Retrieve current points, defaulting to 0 if the key does not exist
                current_points = pipe.get(key)
                if current_points is None:
                    current_points = 0
                else:
                    current_points = int(current_points)
                
                # Begin transaction
                pipe.multi()
                # Add the new points to the current points
                pipe.set(key, current_points + points)
                # Execute transaction
                pipe.execute()
                
                # If successful, break out of the retry loop
                break
        except redis.WatchError:
            # If another client modified the key, retry
            continue

# Test the function by updating points for user 2
client.set('points:2', 200)
update_points(2, 30)

# Retrieve and print the updated points
value = client.get('points:2').decode('utf-8')
print(f"Updated points for user 2: {value}")
```

### Explanation:
1. **Watch the key**: The `pipe.watch(key)` command monitors the `points:{user_id}` key. If another client changes this key after the `watch` command and before the `execute`, a `WatchError` is raised.
2. **Get current points**: The points are fetched with `pipe.get(key)`, and if the key does not exist, it's initialized to 0.
3. **Transaction**: Using `pipe.multi()` begins the transaction, and the new points are added to the existing ones.
4. **Retry mechanism**: The `WatchError` exception is handled in a loop, which ensures that the transaction is retried until it succeeds.

You can now run the script, and it will successfully update the user's points even in the presence of concurrent updates.